## Agent Workflow Exploration

## Import statements and dependencies

In [79]:
from tavily import TavilyClient
import os
from rich import print as rprint
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from math_tools import get_math_tool
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch

## Tool Logic

### Tool 1 - Tavily web search

In [5]:
# base level querying using tavily client

tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))
response = tavily_client.search("Who is Leo Messi?")
rprint(response)


{
    'query': 'Who is Leo Messi?',
    'follow_up_questions': None,
    'answer': None,
    'images': [],
    'results': [
        {
            'title': 'Career of Lionel Messi - Wikipedia',
            'url': 'https://en.wikipedia.org/wiki/Career_of_Lionel_Messi',
            'content': "Lionel Messi is an Argentine professional footballer who plays as a forward for and 
captains both Major League Soccer club Inter Miami and the Argentina national team.His individual achievements 
include eight Ballon d'Or awards, the most for any footballer. Having won 45 team trophies, [note 1] he is the most
decorated player in the history of professional football. [11]",
            'score': 0.9970765,
            'raw_content': None
        },
        {
            'title': 'Lionel Messi | Biography, Competitions, Wins and Medals',
            'url': 'https://olympics.com/en/athletes/lionel-messi',
            'content': '“The Olympic gold in 2008 is the win that I value the most because it is a tournament that 
you may play only once in your life and involves many athletes from different disciplines” - Lionel Messi, Spanish 
Esquire\nFeatured\nLatest News\nOlympic Results\nAthlete Olympic Results Content\nYou may like Lionel 
MESSI\nBiography\nBorn in Rosario, Argentina in 1987 Lionel Messi is widely held to be one of the greatest football
players of his generation — and with good reason.\n Graduating quickly through the club’s ranks thanks to his 
ever-growing skill set Messi made his first senior appearance for the club at the age of 17.\n When he eventually 
left the team in 2021 for French club Paris Saint-Germain the Argentine completed his time in La Liga as its top 
goal-scorer, with an eye-watering 474 successful shots to his name.\n He also came close to winning the FIFA World 
Cup in 2014 but Argentina fell to Germany 1-0 in the final.\n',
            'score': 0.94240075,
            'raw_content': None
        },
        {
            'title': "Who does Lionel Messi play for? What to know about soccer star's team ...",
            'url': 
'https://www.sportingnews.com/us/soccer/news/lionel-messi-team-world-cup-2022-soccer/pyan3de8jqalupyqzzthkax8',
            'content': "Who does Lionel Messi play for in the World Cup?\nMessi is Argentina's No. 10, following in
the footsteps of iconic Argentinian playmaker Diego Maradona.\n However, after a series of financial issues (which 
still persist), Barça were forced to let Messi — the highest-paid player in the world — leave following the 2020-21
season.\n He nabbed another Balon D'Or in 2021 and is continuing to put on a clinic even at the age of 35 — he has 
seven goals and ten assists in 13 games for PSG this season.\n In 2021, Messi led Argentina to a Copa America 
title, meaning that the Argentinians reign supreme as the best team in South America at the moment. And, for likely
the final time, fans will be able to witness him perform on the grand stage: the World Cup.\n",
            'score': 0.93071055,
            'raw_content': None
        },
        {
            'title': "Lionel Messi | Records, Height, Ballon d'Or, Inter Miami, & Facts ...",
            'url': 'https://www.britannica.com/biography/Lionel-Messi',
            'content': 'In early 2009 Messi capped off a spectacular 2008–09 season by helping FC Barcelona capture
the club’s first “treble” (winning three major European club titles in one season): the team won the La Liga 
championship, the Copa del Rey (Spain’s major domestic cup), and the Champions League title. Messi’s play continued
to rapidly improve over the years, and by 2008 he was one of the most dominant players in the world, finishing 
second to Manchester United’s Cristiano Ronaldo in the voting for the 2008 Ballon d’Or. At the 2014 World Cup, 
Messi put on a dazzling display, scoring four goals and almost single-handedly propelling an offense-deficient 
Argentina team through the group stage and into the knockout rounds, where Argentina then adv

In [8]:
# context generation for RAG using tavily client

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
context = tavily_client.get_search_context(query="Who is Leo Messi?")
rprint(context)

"[\"{\\\"url\\\": \\\"https://kids.kiddle.co/Lionel_Messi\\\", \\\"content\\\": \\\"Lionel Andr\\\\u00e9s Messi ( 
born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and
captains both Major League Soccer club Inter Miami and the Argentina national team. Messi has also invested in 
youth football in Argentina: he financially supports Sarmiento, a football club based in the Rosario neighbourhood 
where he was born, committing in 2013 to the refurbishment of their facilities and the installation of all-weather 
pitches, and funds the management of several youth players at Newell's Old Boys and rival club Rosario Central, as 
well as at River Plate and Boca Juniors in Buenos Aires.\\\"}\", \"{\\\"url\\\": 
\\\"https://www.thefamouspeople.com/profiles/lionel-messi-5242.php\\\", \\\"content\\\": \\\"Lionel Messi is an 
Argentine former professional footballer counted amongst the best players in the world in football history. The 
only player in history to win five FIFA Ballons d'Or, he was also the first player to win three European Golden 
Shoes. During his playing career he shattered many world records and currently holds the records for most goals 
scored in La Liga, a La Liga season (50\\\"}\", \"{\\\"url\\\": 
\\\"https://www.britannica.com/biography/Lionel-Messi\\\", \\\"content\\\": \\\"In early 2009 Messi capped off a 
spectacular 2008\\\\u201309 season by helping FC Barcelona capture the club\\\\u2019s first 
\\\\u201ctreble\\\\u201d (winning three major European club titles in one season): the team won the La Liga 
championship, the Copa del Rey (Spain\\\\u2019s major domestic cup), and the Champions League title. 
Messi\\\\u2019s play continued to rapidly improve over the years, and by 2008 he was one of the most dominant 
players in the world, finishing second to Manchester United\\\\u2019s Cristiano Ronaldo in the voting for the 2008 
Ballon d\\\\u2019Or. At the 2014 World Cup, Messi put on a dazzling display, scoring four goals and almost 
single-handedly propelling an offense-deficient Argentina team through the group stage and into the knockout 
rounds, where Argentina then advanced to the World Cup final for the first time in 24 years. After Argentina was 
defeated in the Copa final\\\\u2014the team\\\\u2019s third consecutive finals loss in a major 
tournament\\\\u2014Messi said that he was quitting the national team, but his short-lived 
\\\\u201cretirement\\\\u201d lasted less than two months before he announced his return to the Argentine team. 
Messi helped Barcelona capture another treble during the 2014\\\\u201315 season, leading the team with 43 goals 
scored over the course of the campaign, which resulted in his fifth world player of the year honour.\\\"}\", 
\"{\\\"url\\\": \\\"https://en.wikipedia.org/wiki/Lionel_Messi\\\", \\\"content\\\": \\\"He scored twice in the 
last group match, a 3\\\\u20132 victory over Nigeria, his second goal coming from a free kick, as they finished 
first in their group.[423] Messi assisted a late goal in extra time to ensure a 1\\\\u20130 win against Switzerland
in the round of 16, and played in the 1\\\\u20130 quarter-final win against Belgium as Argentina progressed to the 
semi-final of the World Cup for the first time since 1990.[424][425] Following a 0\\\\u20130 draw in extra time, 
they eliminated the Netherlands 4\\\\u20132 in a penalty shootout to reach the final, with Messi scoring his team's
first penalty.[426]\\\\nBilled as Messi versus Germany, the world's best player against the best team, the final 
was a repeat of the 1990 final featuring Diego Maradona.[427] Within the first half-hour, Messi had started the 
play that led to a goal, but it was ruled offside. \\\\\\\"[582] Moreover, several pundits and footballing figures,
including Maradona, questioned Messi's leadership with Argentina at times, despite his playing 
ability.[583][584][585] Vickery states the perception of Messi among Argentines chan

In [9]:
# q&a using tavily client

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
answer = tavily_client.qna_search(query="Who is Leo Messi?")
rprint(answer)

Lionel Messi, born in Rosario, Argentina in 1987, is widely considered one of the greatest football players of his 
generation. Graduating through the ranks of FC Barcelona, Messi became their top goal-scorer before moving to Paris
Saint-Germain in 2021. He has won multiple titles, including the Olympic gold in 2008 and has been a dominant force
in both club and international football, with impressive goal-scoring records in both realms.

In [77]:
llm = ChatOpenAI(model="gpt-4-turbo")
search = TavilySearchResults(
    max_results=2,  # Increase the number of results for better context
    description='tavily_search_results_json(query="the search query") - a search engine.',
)
start_query = "Who is Leo Messi and what is his father's last name and age?"

prompt = (
        f"You are tasked with answering the following: {start_query}. "
        "Generate more questions you may need answers to in order to answer this questions and return all separated only by a single comma with no spaces."
        "If you do not need anymore context, just return the query."
        "This sub-query will be passed to a search engine to provide more context to answer this query."
        "Ensure that you are using information that is the most up to date."
        "If you are unsure, just return the query. Do not try to guess."
    )

generated_query = llm.invoke(prompt)

rprint("The genereated query is:")
rprint(generated_query)

context = ""

for query in generated_query.content.split(","):
    search_results = search.invoke({"query": query.strip().strip('"')})
    rprint(f"Search result for the query: {query} is...")
    rprint(search_results)
    if search_results:
        for search_result in search_results:
            context += (search_result['content']) + "\n"

rprint("The collated context for all the sub-queries is...")
rprint(context)

final_prompt = (
        f"Context:\n{context}\n\n"
        f"Question:\n{start_query}\n"
        "Please answer the question based on the provided context."
    )

result = llm.invoke(final_prompt)

rprint(f"This is the answer to the query: {start_query}")
rprint(result)

The genereated query is:

AIMessage(
    content="Who is Leo Messi,What is Leo Messi's father's last name,How old is Leo Messi's father",
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 22,
            'prompt_tokens': 116,
            'total_tokens': 138,
            'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
            'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4-turbo-2024-04-09',
        'system_fingerprint': 'fp_ed14c08274',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-ea67ec36-550e-400f-8057-3c0881e77fe9-0',
    usage_metadata={
        'input_tokens': 116,
        'output_tokens': 22,
        'total_tokens': 138,
        'input_token_details': {'cache_read': 0},
        'output_token_details': {'reasoning': 0}
    }
)

Search result for the query: Who is Leo Messi is...

[
    {
        'url': 'https://olympics.com/en/athletes/lionel-messi',
        'content': '“The Olympic gold in 2008 is the win that I value the most because it is a tournament that you 
may play only once in your life and involves many athletes from different disciplines” - Lionel Messi, Spanish 
Esquire\nFeatured\nLatest News\nOlympic Results\nAthlete Olympic Results Content\nYou may like Lionel 
MESSI\nBiography\nBorn in Rosario, Argentina in 1987 Lionel Messi is widely held to be one of the greatest football
players of his generation — and with good reason.\n Graduating quickly through the club’s ranks thanks to his 
ever-growing skill set Messi made his first senior appearance for the club at the age of 17.\n When he eventually 
left the team in 2021 for French club Paris Saint-Germain the Argentine completed his time in La Liga as its top 
goal-scorer, with an eye-watering 474 successful shots to his name.\n He also came close to winning the FIFA World 
Cup in 2014 but Argentina fell to Germany 1-0 in the final.\n'
    },
    {
        'url': 'https://www.nytimes.com/athletic/4679809/2023/07/19/lionel-messi-evolution-barcelona-argentina/',
        'content': "Lionel Messi profile: The life and times of Inter Miami's World Cup winner; The Dribbling 
Winger. Next time you've got a minute to reconsider whether you've ever truly been good at anything,"
    }
]

Search result for the query: What is Leo Messi's father's last name is...

...

[
    {
        'url': 
'https://www.nytimes.com/athletic/2029445/2020/09/02/who-is-jorge-messi-leo-bartomeu-barcelona-transfer/',
        'content': 'Meet Jorge Messi, Leo\'s father, who is set for a meeting with Bartomeu in maybe the most high 
profile football agent-president summit ever ... "From when he was 13 years old, until now, his'
    },
    {
        'url': 'https://www.sportsdave.com/biography-of-jorge-messi-lionel-messis-father/',
        'content': 'Biography. Jorge Messi was born on January 1, 1958, in Argentine to his parent Eusebio Messi, 
and Rosa Maria Pérez. He gained fame as the father of legendary Argentine footballer Lionel Messi. He is the 
great-grandson of immigrants from Catalonia and the northwest Adriatic Marche region of Italy. He has both Italian 
and Spanish ancestry.'
    }
]

The collated context for all the sub-queries is...

“The Olympic gold in 2008 is the win that I value the most because it is a tournament that you may play only once 
in your life and involves many athletes from different disciplines” - Lionel Messi, Spanish Esquire
Featured
Latest News
Olympic Results
Athlete Olympic Results Content
You may like Lionel MESSI
Biography
Born in Rosario, Argentina in 1987 Lionel Messi is widely held to be one of the greatest football players of his 
generation — and with good reason.
 Graduating quickly through the club’s ranks thanks to his ever-growing skill set Messi made his first senior 
appearance for the club at the age of 17.
 When he eventually left the team in 2021 for French club Paris Saint-Germain the Argentine completed his time in 
La Liga as its top goal-scorer, with an eye-watering 474 successful shots to his name.
 He also came close to winning the FIFA World Cup in 2014 but Argentina fell to Germany 1-0 in the final.

Lionel Messi profile: The life and times of Inter Miami's World Cup winner; The Dribbling Winger. Next time you've 
got a minute to reconsider whether you've ever truly been good at anything,
All About Lionel Messi's Parents, Jorge Messi and Celia Cuccittini
Lionel Messi has a close relationship with his dad and mom
Jorge Messi Instagram
Lionel Messi's parents, Jorge Messi and Celia Cuccittini, played a pivotal role in shaping their son’s career. 
Jorge has guided Lionel's career from the beginning
Jorge Messi Instagram
The Argentinian footballer has had one of the most impressive careers in the history of sports, and it all started 
with his dad.
 “He always played for fun and we were happy to see him play not because we thought he’d be a triumphant success 
but simply because he enjoyed it and did it well.”
Reflecting on encouraging her son in the World Cup, Celia opened up on the TV show El Diario de Mariana in 2018.
 "
They are protective of Lionel
Jorge Messi Instagram
Jorge and Celia are incredibly supportive of their youngest son, and Jorge often comes to Lionel’s defense when he 
believes his son is being unfairly criticized. “I always say the same: what makes me proud is who Leo is as a 
person,” Jorge shared in Bruno Pisano’s 2011 book, My Son The Soccer Player: The Secrets of the World’s Greatest 
Players as Told by Their Parents, per SoccerAmerica.

| The US Sun MORE FROM THE SUN BEFORE Lionel Messi was one of the greatest football players of all time, he was 
only known as the son of Jorge Messi and Celia María Cuccittini. Lionel Messi is the son of Jorge and CeliaCredit: 
Instagram/ jorge.sole When did Jorge Messi become Lionel's agent? While Lionel might be the most well-known Messi 
family member, he is not Jorge and Celia's only child. Jorge and Celia are also the parents of Lionel's older 
brothers, Rodrigo and Matias, and his younger sister, Maria Sol. While the Messi kids might have a high-profile 
brother, they often stay out of the spotlight as much as possible. More from The Sun
Meet Jorge Messi, Leo's father, who is set for a meeting with Bartomeu in maybe the most high profile football 
agent-president summit ever ... "From when he was 13 years old, until now, his
Biography. Jorge Messi was born on January 1, 1958, in Argentine to his parent Eusebio Messi, and Rosa Maria Pérez.
He gained fame as the father of legendary Argentine footballer Lionel Messi. He is the great-grandson of immigrants
from Catalonia and the northwest Adriatic Marche region of Italy. He has both Italian and Spanish ancestry.

This is the answer to the query: Who is Leo Messi and what is his father's last name and age?

AIMessage(
    content="Leo Messi, born Lionel Messi, is widely regarded as one of the greatest football players of his 
generation. He was born in Rosario, Argentina, in 1987 and had a prolific career with FC Barcelona before moving to
Paris Saint-Germain and then Inter Miami. His father's name is Jorge Messi, who was born on January 1, 1958, making
him 65 years old as of 2023.",
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 87,
            'prompt_tokens': 827,
            'total_tokens': 914,
            'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
            'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4-turbo-2024-04-09',
        'system_fingerprint': 'fp_ed14c08274',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-54436cbb-5141-4b7d-b94e-11e33561951c-0',
    usage_metadata={
        'input_tokens': 827,
        'output_tokens': 87,
        'total_tokens': 914,
        'input_token_details': {'cache_read': 0},
        'output_token_details': {'reasoning': 0}
    }
)

### Tool 2 - Math

In [70]:
# Initialize the LLM and tools
llm = ChatOpenAI(model="gpt-4-turbo")
calculate = get_math_tool(llm)
search = TavilySearchResults(
    max_results=3,  # Increase the number of results for better context
    description='tavily_search_results_json(query="the search query") - a search engine.',
)

# Example query
query = "What is the age of Leo Messi in 2010 divided by 5?"

# Step 1: Prepare the query for the LLM
prompt = (
    f"You are tasked with calculating the following: {query}. "
    "Return a sub-query to search for any relevant information you may need to answer the query."
    "This sub-query will be passed to a search engine to provide more context to answer this query."
    "Ensure that you are using information that is most up to date."
    "If you are unsure, just return the query. Do not try to guess."
)

# Step 2: Invoke the LLM to get the search query
generated_query = llm.invoke(prompt)

rprint("Generated query for Tavily:")
rprint(generated_query)

# Step 3: Perform the search using the generated query
search_results = search.invoke({"query": generated_query.content.strip().strip('"')})

rprint("Search results from Tavily:")
rprint(search_results)

# Step 4: Extract relevant context if search results are available
context = ""
if search_results:
    for result in search_results:
        context += f"{result['content']} "

context = f"{context}"

rprint("This is the context to be passed to the LLM")
rprint(context)

# Step 5: Prepare input for the calculation tool
input_data = {
    "problem": f"{query}.",
    "context": [context],  # Include filtered and summarized context for the calculation
    "query": generated_query.content.strip().strip('"')  # Pass the generated query as well
}

# Step 6: Invoke the calculate tool
result = calculate.invoke(input_data)

rprint(f"This is the answer to the query: {query}")
rprint(result)

Generated query for Tavily:

AIMessage(
    content='Leo Messi birth year',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 4,
            'prompt_tokens': 95,
            'total_tokens': 99,
            'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
            'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4-turbo-2024-04-09',
        'system_fingerprint': 'fp_4c7a0da424',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-c54e4a68-1f06-4ecf-8e1d-aaf3679fc2bc-0',
    usage_metadata={
        'input_tokens': 95,
        'output_tokens': 4,
        'total_tokens': 99,
        'input_token_details': {'cache_read': 0},
        'output_token_details': {'reasoning': 0}
    }
)

Search results from Tavily:

[
    {
        'url': 'https://birthdaywiki.com/messi',
        'content': "Messi was born with the birth name Lionel Andres Messi. When and Where was Messi born? ... From
the year 1994, Messi began his professional career. He used to play football for Newell's Old Boys and then he 
played for Newell's till 2000. After that, he joined Barcelona in the year 2001."
    },
    {
        'url': 'https://starktimes.com/leo-messi/',
        'content': "Leo Messi Age, Height, Weight. As per his date of birth Leo Messi's age is 33 years. Talking 
about his physical appearance Leo Messi's height is 5′ 7″ and his weight is approx 74 kg.He is a handsome and 
attractive man with brown eyes and brown hair."
    },
    {
        'url': 'https://www.transfermarkt.com/lionel-messi/profil/spieler/28003',
        'content': 'Lionel Messi, 37, from Argentina Inter Miami CF, since 2023 Right Winger Market value: €25.00m 
* Jun 24, 1987 in Rosario, Argentina ... Name in home country: Lionel Andrés Messi Cuccitini Date of birth/Age: Jun
24, 1987 (37) Place of birth:'
    }
]

This is the context to be passed to the LLM

Messi was born with the birth name Lionel Andres Messi. When and Where was Messi born? ... From the year 1994, 
Messi began his professional career. He used to play football for Newell's Old Boys and then he played for Newell's
till 2000. After that, he joined Barcelona in the year 2001. Leo Messi Age, Height, Weight. As per his date of 
birth Leo Messi's age is 33 years. Talking about his physical appearance Leo Messi's height is 5′ 7″ and his weight
is approx 74 kg.He is a handsome and attractive man with brown eyes and brown hair. Lionel Messi, 37, from 
Argentina Inter Miami CF, since 2023 Right Winger Market value: €25.00m * Jun 24, 1987 in Rosario, Argentina ... 
Name in home country: Lionel Andrés Messi Cuccitini Date of birth/Age: Jun 24, 1987 (37) Place of birth:

This is the answer to the query: What is the age of Leo Messi in 2010 divided by 5?

4.6

### Tool 3 - RAG

In [89]:
client = MongoClient(os.getenv("MONGODB_URI"))
dbName = "atlas_exploration"
collectionName = "chunked_data"
index = "vector_index"
collection = client[dbName][collectionName]

vectorStoreRetriever = MongoDBAtlasVectorSearch.from_connection_string(
    os.getenv("MONGODB_URI"),
    dbName + "." + collectionName,
    OpenAIEmbeddings(disallowed_special=(), openai_api_key=os.getenv("OPENAI_API_KEY")),
    index_name=index,
)

retriever = vectorStoreRetriever.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3, "pre_filter": {"hasCode": {"$eq": False}}},
    )

rag_query = "What are the charges of an ILP?"

results = retriever.invoke(rag_query)

rprint("These are the results of the vector search")
rprint(results)

context = ""
for result in results:
    context += result.page_content + '\n'

llm = ChatOpenAI(model="gpt-4-turbo")
prompt = (
        f"Context:\n{context}\n\n"
        f"Question:\n{rag_query}\n"
        "Please answer the question based on the provided context."
    )

result = llm.invoke(prompt)

rprint(f"This is the answer to the query: {rag_query}")
rprint(result)

These are the results of the vector search

[
    Document(
        metadata={
            '_id': '671c7fd2d979057b1f309f27',
            'title': ['Module 9A: Life Insurance and Investment -Linked Policies II'],
            'hasCode': False,
            'source': '../data/M9A_3rd_Edition.pdf',
            'page': 109
        },
        page_content='as those describe d below .  \n \nB1. Fees And C harges  \n \n6.11 There are two ways to pay 
fees and charges under an  ILP. One way is through \ndeduction of premium or cancellation of  units. For example, 
the upfront sales \ncommission is deducted from the single premium before the balance is invested i n \n \n7   
Refer to Notice No: MAS 307 , Annex Ha  for the prescribed template.'
    ),
    Document(
        metadata={
            '_id': '671c7fd2d979057b1f309ecd',
            'title': ['Introduction to Structured ILPs'],
            'hasCode': False,
            'source': '../data/M9A_3rd_Edition.pdf',
            'page': 98
        },
        page_content='2.8 Structured ILPs share some common drawbacks with conventional ILPs, as \nhighlighted 
below.  \n  \nB1. Fees And C harges  \n \n2.9 An ILP has several layers of expenses.   \n\uf0a7 Front -end sales 
charge, typically 5% for single premium, and c an be as high as \n50% to 80% of first year premium for regular 
premium policies.       \n \n\uf0a7 Annual fund management fee, based on the size of the fund and asset types, 
\ntypically 1% to 2% for equities , and 0.5% to 1% for fixed incomes. A general rule'
    ),
    Document(
        metadata={
            '_id': '671c7fd2d979057b1f309f15',
            'title': ['Module 9A: Life Insurance and Investment -Linked Policies II'],
            'hasCode': False,
            'source': '../data/M9A_3rd_Edition.pdf',
            'page': 107
        },
        page_content='state so and indicate its risk classification.  \n   \n(ii) Fees And C harges  \n 
\nDisclosure of all fees and charges for the ILP and ILP sub- fund, whether \nthrough deduction from premium, 
cancellation of units, or deduction from \nasset value of the ILP sub- fund. Where there is a provision for a 
maximum fee \nor charge payable, highlight that fact and state that maximum.  \n \n(iii) Dealing of Units  \n 
\nState how units in the ILP sub- fund may be purchased and paid for; redeemed'
    )
]

This is the answer to the query: What are the charges of an ILP?

AIMessage(
    content="The charges of an Investment-Linked Policy (ILP) include several types of fees and expenses, as 
outlined in the provided context:\n\n1. **Front-end Sales Charge**: This is a charge applied at the time of your 
initial investment. For single premiums, this charge is typically around 5%. For regular premium policies, this 
charge can be significantly higher, ranging from 50% to 80% of the first year's premium.\n\n2. **Annual Fund 
Management Fee**: This fee is charged for the management of the funds in which your premiums are invested. The rate
depends on the type of assets managed. Typically, it ranges from 1% to 2% for equities and 0.5% to 1% for fixed 
incomes.\n\nThese fees can be deducted in various ways, including from the premium paid, from the cancellation of 
units, or directly from the asset value of the ILP sub-fund. Additionally, if there is a maximum limit on the fees 
or charges that can be applied, this must be clearly stated.",
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 211,
            'prompt_tokens': 371,
            'total_tokens': 582,
            'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
            'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4-turbo-2024-04-09',
        'system_fingerprint': 'fp_ed14c08274',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-9e724670-9419-4ff0-bf27-7cf7365675a5-0',
    usage_metadata={
        'input_tokens': 371,
        'output_tokens': 211,
        'total_tokens': 582,
        'input_token_details': {'cache_read': 0},
        'output_token_details': {'reasoning': 0}
    }
)

### Multi Agent Workflow with Tools

#### Web Search Tool Node

#### Math Tool Node

#### Rag Tool Node